# Normalizer Performance Analysis

This notebook contains an analysis of the normalizer performance on the CIViC, MOA, and Clinvar data

## import relevant packages

In [2]:
from pathlib import Path
import pandas as pd
# from civicpy import civic as civicpy
# import plotly.express as px
import ndjson
import re
from dotenv import load_dotenv
import os

## Dictionaries to hold categories and df to hold variant counts.

In [3]:
moa_category_bins = {
    "Sequence Variants": "Sequence Variants",
    "Copy Number Variants": "Copy Number Variants"
}

## CIViC

Read CIViC .csv files

In [6]:
os.path.realpath("merged_performance_analysis.ipynb")
os.chdir('/Users/dpp002/Documents/Variant Normalizer Manuscript/variation-normalizer-manuscript-1/analysis')
print("Current working directory: {0}".format(os.getcwd()))
for entry in os.scandir('.'):
    if entry.is_file():
        print(entry.name)

Current working directory: /Users/dpp002/Documents/Variant Normalizer Manuscript/variation-normalizer-manuscript-1/analysis
.DS_Store
clinvar_analysis.ipynb
merged_moa_civic_evidence_analysis.ipynb


In [5]:
os.chdir('/Users/dpp002/Documents/Variant Normalizer Manuscript/variation-normalizer-manuscript-1/analysis/civic/variation_analysis')
print("Current working directory: {0}".format(os.getcwd()))

for entry in os.scandir('.'):
    if entry.is_file():
        print(entry.name)

Current working directory: /Users/dpp002/Documents/Variant Normalizer Manuscript/variation-normalizer-manuscript-1/analysis/civic/variation_analysis
unable_to_normalize_queries.csv
not_supported_variants.csv
should_be_able_to_normalize_genomic_variant_queries.csv
should_be_able_to_normalize_protein_variant_queries.csv
able_to_normalize_queries.csv
civic_variation_analysis.ipynb


In [ ]:
# civic_supported_df = 

Get CIViC Variant counts by category, update variant counts df 

## MOA

Read MOA .csv files

In [32]:
moa_normalized_df = pd.read_csv("/Users/dpp002/Documents/Variant Normalizer Manuscript/variation-normalizer-manuscript-1/analysis/moa/feature_analysis/able_to_normalize_queries.csv",sep = "\t")
print(moa_normalized_df.shape)
moa_normalized_df.head()
type(moa_normalized_df)

(181, 5)


pandas.core.frame.DataFrame

In [42]:
moa_normalized_category_counts = moa_normalized_df["category"].value_counts(dropna=False)
# moa_normalized_category_counts = moa_normalized_df["category"].value_counts(dropna=False).reset_index().rename(columns={"index": "category", 0: "count"})
moa_normalized_category_counts.head()
indeces = moa_normalized_category_counts.index
print(indeces)
print(indeces[0],',',moa_normalized_category_counts[0])
print(indeces[1],',',moa_normalized_category_counts[1])


Index(['Sequence Variants', 'Copy Number Variants'], dtype='object', name='category')
Sequence Variants , 149
Copy Number Variants , 32


Get MOA Variant counts by category, update variant counts df 

## ClinVar

Get df from ClinVar Analysis

Get Clinvar Variant counts by category, update variant counts df 

Output counts df

Generate figure(?)